In [15]:
import cv2
import numpy as np

## background removing from video

In [11]:
cap = cv2.VideoCapture(0)

algo1 = cv2.createBackgroundSubtractorKNN()
algo2 = cv2.createBackgroundSubtractorMOG2()

while True:
    r,f = cap.read()
    if r == True:
        f = cv2.resize(f,(500,600))
        r1 = algo1.apply(f)
        r2 = algo2.apply(f)
        cv2.imshow("window1", r1)
        cv2.imshow("window2",r2)
        cv2.imshow("ws",f)
        if cv2.waitKey(25) & 0xff == ord("p"):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)
        

-1

## object tracking

In [36]:
cap = cv2.VideoCapture(0)

r,f = cap.read()
x,y,w,h =220, 150, 120, 350 ##  Sets a hardcoded rectangle (x, y, width, height) as the region of interest (ROI).
t = (x,y,w,h)  ## track
roi = f[y:y+h, x:x+w]
hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)     ## Converts it to HSV color space (which is better for color segmentation
mask = cv2.inRange(hsv_roi, np.array((0.,60.,32.)), np.array((180.,255.,255.)))  # filter out irrelevant background
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[100],[0,180])    ## filter it using histogram
##In object tracking, we want to recognize the object in future frames. 
##One simple and effective way is to track based on color. 
##A histogram gives us a statistical representation of color distribution.


cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)      ## we have normalised it

tr = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,1)  ## Termination criteria when to stop
##cv2.TERM_CRITERIA_EPS – stop the algorithm if the movement of the window is smaller than epsilon.

##cv2.TERM_CRITERIA_COUNT – stop after a maximum number of iterations.

while True:
    r,f = cap.read()
    if r == True:
        f = cv2.resize(f,(500,600))
        hsv_f = cv2.cvtColor(f, cv2.COLOR_BGR2HSV)
        d = cv2.calcBackProject([hsv_f],[0],roi_hist,[0,180],1)  ## here we track the image
        r,tp = cv2.meanShift(d,t,tr)              ## it basically gives us the coordinate of the shifted image
        x,y,w,h= tp
        final = cv2.rectangle(f,(x,y),(x+w, y+h),(0,0,255),4)
        cv2.imshow("window", final)
        if cv2.waitKey(25) & 0xff == ord("p"):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)
        

-1

## Detect the corner points   cornerHarris

In [45]:
img = cv2.imread("img1.jpg")
gr = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

gr = np.float32(gr)  ## required float input

res = cv2.cornerHarris(gr,2,3,0.04)  ##(img, blocksize, mat.size, parameter) res: A matrix of the same size as the image, 
                        ##where corner strength is stored in each pixel. Higher values mean more "corner-like".

res = cv2.dilate(res,None)  ## its a filter enhance the corner points
img[res>0.01*res.max()]=  [0,0,255]   ## strong corners where val is 1% of max value

cv2.imshow("window",img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

## corner detection using shi-tomasi corner detection

In [51]:
img = cv2.imread("img1.jpg")
gr = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cr = cv2.goodFeaturesToTrack(gr, 10, 0.01, 20) ### img, maxcorner_points, qualityLevel, mindistance,

cr = np.int64(cr)                 ## gives us float values -> to int
print(cr)

for i in cr:
    x,y = i.ravel()                        ## convert data in 1D
    cv2.circle(img, (x,y), 5, (0,0,255), -1)  ## -1 solid thickness
    


cv2.imshow("window",img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

[[[316 446]]

 [[172 507]]

 [[214 261]]

 [[288 345]]

 [[180 442]]

 [[195 121]]

 [[157 111]]

 [[167 403]]

 [[166  30]]

 [[ 38 489]]]


-1